<a href="https://colab.research.google.com/github/strongeryoung/class_SmartDataAnalysis/blob/main/%EC%B6%94%EA%B0%80_%EB%AA%85%EB%A0%B9%EC%96%B4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install webdriver_manager
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager
service = ChromeService(executable_path=ChromeDriverManager().install())

In [ ]:
wb=openpyxl.load_workbook('list.xlsx')
ws=wb.active

no_row=ws.max_row
print("행갯수는:",no_row)

company_sequences=[]
for i in range(2,no_row+1):
    value=ws.cell(row=i,column=2).value # 신문사의 크롤링 여부를 value에 저장
    if value=="O": #value가 O이면 리스트에 저장
        company_sequences.append(i-2)
print("크롤링할 신문사의 순서 리스트:",company_sequences)

start_date=ws.cell(row=2,column=3).value # 엑셀에서 시작일 읽어오기
end_date=ws.cell(row=2,column=4).value # 엑셀에서 종료일 읽어오기

print("시작일은:",start_date)
print("종료일은:",end_date)

keyword_list=[]
for i in range(2,no_row+1):
    value=ws.cell(row=i,column=5).value # 신문사의 크롤링 여부를 value에 저장
    if value!="" and value!=None : #value가 O이면 리스트에 저장
        keyword_list.append(value)
print("키워드리스트는:",keyword_list)



chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]  # 크롬 버전을 확인한다.
driver_path = r'여러분의 경로'
if os.path.exists(driver_path):
    print(f"chromedriver is installed: {driver_path}")  # 있는 버전을 쓴다.
else:
    print(f"install the chrome driver(ver: {chrome_ver})")  # 크롬을 최신 버전으로 설치한다.
    chromedriver_autoinstaller.install(True)

options = webdriver.ChromeOptions()  # 크롬 옵션을 추가한다.
# options.add_argument('headless')
options.add_experimental_option("detach", True)  # 크롬 안 꺼지는 옵션 추가
options.add_experimental_option("excludeSwitches", ["enable-logging"])  # 크롬 안 꺼지는 옵션 추가
browser = webdriver.Chrome(service=service, options=options)  # 크롬 드라이버를 할당
url = 'https://www.bigkinds.or.kr/'  # 로그인 사이트 접속
browser.get(url)
browser.maximize_window()
browser.implicitly_wait(3)

btn_login=browser.find_element(By.CSS_SELECTOR,'#header > div.hd-top > div > div > button.btn-login.login-modal-btn.login-area-before') #로그인버튼 클릭
browser.execute_script("arguments[0].click();", btn_login)  # 로그인버튼을 찾아 클릭한다!!
time.sleep(2)

input_id=browser.find_element(By.CSS_SELECTOR,'#login-user-id') #id입력
input_id.send_keys("여러분의 아이디")

time.sleep(0.5)
input_pw=browser.find_element(By.CSS_SELECTOR,'#login-user-password') #pw입력
input_pw.send_keys("여러분의 비번")

btn_login=browser.find_element(By.CSS_SELECTOR,'#login-btn') # 로그인 클릭
browser.execute_script("arguments[0].click();", btn_login)  #
browser.implicitly_wait(3)
time.sleep(1)


url = 'https://www.bigkinds.or.kr/v2/news/index.do'  # 검색 사이트로 이동
browser.get(url)
browser.maximize_window()
browser.implicitly_wait(3)
company_group=browser.find_element(By.CLASS_NAME,'chk-lst')
company_list=company_group.find_elements(By.TAG_NAME,'li')
for index,company in enumerate(company_list): # 신문사를 엑셀 정보를 파악하여 선택한다.
    for company_sequence in company_sequences:
        if company_sequence==index:
            company.click()
    time.sleep(0.2)
time.sleep(1)

btn_period = browser.find_element(By.CSS_SELECTOR, '#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp1 > div.tab-btn-inner.tab1 > a') #기간선택을 한다.
browser.execute_script("arguments[0].click();", btn_period)  #
time.sleep(1)

print("시작일 입력")
btn_start_date = browser.find_element(By.CSS_SELECTOR, '#srch-tab1 > div > div.item-input > div > div:nth-child(1)') #시작일을 입력한다.
btn_start_date.click()
ActionChains(browser).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
ActionChains(browser).send_keys(Keys.DELETE).perform()
ActionChains(browser).send_keys(start_date).perform()
time.sleep(1)

print("종료일 입력")
btn_end_date = browser.find_element(By.CSS_SELECTOR, '#srch-tab1 > div > div.item-input > div > div:nth-child(3)') #종료일을 입력한다.
btn_end_date.click()
ActionChains(browser).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform()
ActionChains(browser).send_keys(Keys.DELETE).perform()
ActionChains(browser).send_keys(end_date).perform()
time.sleep(1)


print("검색옵션 설정")
btn_search_condition = browser.find_element(By.CSS_SELECTOR, '#collapse-step-1-body > div.srch-detail.v2 > div > div.tab-btn-wp3 > div.tab-btn-inner.tab5 > a') #검색 옵션을 선택한다.(ONLY 제목)
browser.execute_script("arguments[0].click();", btn_search_condition)  #
time.sleep(1)


only_title=browser.find_element(By.CSS_SELECTOR,'#search-scope-type > option:nth-child(2)')
only_title.click()
time.sleep(1)

for keyword in keyword_list: #키워드 별로 검색을 시작한다.
    browser.execute_script('window.scrollTo(0,0)')
    input_search = browser.find_element(By.CSS_SELECTOR, '#total-search-key') # 입력란 선택
    input_search.click()
    ActionChains(browser).key_down(Keys.CONTROL).send_keys('a').key_up(Keys.CONTROL).perform() # 전체 선택
    time.sleep(0.5)
    ActionChains(browser).send_keys(Keys.DELETE).perform() # 삭제 입력
    time.sleep(0.5)
    input_search.send_keys(keyword) #검색어 입력
    time.sleep(0.5)
    input_search.send_keys(Keys.ENTER) #엔터키 입력
    time.sleep(3)

    print("STEP3클릭")
    btn_step3=browser.find_element(By.CSS_SELECTOR,'#contents > div.page-body.no-padding.collap-lst > div > div:nth-child(3)') #STEP3을 선택
    btn_step3.click()
    time.sleep(5)

    print("엑셀 다운로드 클릭")
    btn_save=browser.find_element(By.CSS_SELECTOR,'#analytics-data-download > div.btm-btn-wrp > button') # 엑셀 다운로드 버튼 클릭
    browser.execute_script("arguments[0].click();", btn_save)  #
    time.sleep(1)

    browser.execute_script('window.scrollTo(0,0)') #스크롤 최상단 이동
    btn_search = browser.find_element(By.CSS_SELECTOR,'#contents > div.page-body.no-padding.collap-lst > div > div:nth-child(1)') #STEP01 다시 선택
    btn_search.click()
    time.sleep(3)

# browser.execute_script('window.scrollTo(0,document.body.scrollHeight)') #끝까지 스크롤 내리기